In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import TimeSeriesSplit
import statsmodels.api as sm
import numpy as np

In [3]:
data = pd.read_csv('non_nan.csv', )
data

,Date,unemployment_rate,CPI,treasury_yield,GDP_growth,SP500_return,AZN,BMY,JNJ,LLY,MRK,NVO,NVS,PFE,ROG,inflation_change,unemp_change,treasury_yield_change
0,2000-02-01,4.1,170.000,6.661000,0.496560,-1.522563,-12.828964,-13.228004,-16.339821,-11.121498,-21.701151,2.220031,3.838386,-11.226228,54.440789,1.000,-0.1,-0.141500
1,2000-03-01,4.0,171.000,6.519500,0.511258,9.413333,22.264136,-0.218329,-2.079067,5.804243,0.913712,8.390897,6.420237,14.101954,6.922258,1.000,-0.1,-0.141500
2,2000-04-01,3.8,170.900,6.256522,1.327803,-3.266805,5.567379,-8.205683,17.437698,23.153694,12.400712,-0.097663,2.559423,15.213674,7.370518,-0.100,-0.2,-0.262978
3,2000-05-01,4.0,171.200,5.990526,-0.181797,-1.572223,-0.148357,5.395746,8.484832,-1.296597,7.374072,20.863985,5.169310,5.638019,-8.163265,0.300,0.2,-0.265995
4,2000-06-01,4.0,172.200,6.440455,0.305565,1.728613,10.549735,5.788826,14.239888,31.641749,3.078671,2.813690,8.474599,8.076012,13.131313,1.000,0.0,0.449928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,2023-01-01,3.4,300.536,3.616190,0.390254,6.776820,-3.584079,0.972908,-7.489384,-5.928822,-2.549213,2.541749,-0.110227,-13.817335,16.968326,1.546,-0.1,-0.274810
276,2023-02-01,3.6,301.648,3.531500,-0.442183,-2.514271,-0.290649,-4.328217,-6.217115,-9.568502,-1.089288,1.592445,-7.172811,-7.286115,5.451681,1.112,0.2,-0.084690
277,2023-03-01,3.5,301.808,3.746842,-0.442183,3.313488,8.035329,0.507544,1.862736,10.703390,0.141189,12.873250,9.367574,0.566924,11.025813,0.160,-0.1,0.215342
278,2023-04-01,3.4,302.918,3.663043,-0.442183,1.985238,5.489119,-3.664707,5.612908,15.269915,9.289214,5.836894,16.334413,-4.681371,-1.517467,1.110,-0.1,-0.083799


In [4]:
def create_lags_cov(data, covs, period):
    lagged_covs = []  # List to store the names of lagged columns

    for covariate in covs:
        for i in period:
            lagged_column_name = f'{covariate}_lag{i}'
            data[lagged_column_name] = data[covariate].shift(i)
            lagged_covs.append(lagged_column_name)
    
    data.dropna(inplace=True)
    data = data.reset_index(drop=True)

    return data, lagged_covs

In [5]:
def create_lags_returns(data, tickers, period):

    lagged_tickers = []  # List to store the names of lagged columns

    for tick in tickers:
        for i in period:
            lagged_column_name = f'{tick}_lag{i}'
            data[lagged_column_name] = data[tick].shift(i)
            lagged_tickers.append(lagged_column_name)

    data.dropna(inplace=True)
    data = data.reset_index(drop=True)

    return data, lagged_tickers

In [6]:
# Creating lagged variables
tickers = ['AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO', 'NVS', 'PFE','ROG']
covs = ['unemployment_rate', 'CPI', 'treasury_yield', 'GDP_growth', 'SP500_return', 'inflation_change', 'unemp_change', 'treasury_yield_change']
period_covs = [1, 2, 3]
period_returns = [1,2]

data, lagged_covs = create_lags_cov(data, covs, period=period_covs)
data, lagged_tickers = create_lags_returns(data, tickers, period= period_returns)

# Creating dummy variables
data['Quater_dummy'] = 0
data['Quater_rippel'] = 0
data['Date'] = pd.to_datetime(data['Date'])
for index, row in data.iterrows():
    if row['Date'].month in [3, 6, 9, 12]:
        data.loc[index, 'Quater_dummy'] = 1
    if row['Date'].month in [1, 4, 7, 10]:
        data.loc[index, 'Quater_dummy'] = 1

In [7]:
data.columns

Index(['Date', 'unemployment_rate', 'CPI', 'treasury_yield', 'GDP_growth',
       'SP500_return', 'AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO', 'NVS', 'PFE',
       'ROG', 'inflation_change', 'unemp_change', 'treasury_yield_change',
       'unemployment_rate_lag1', 'unemployment_rate_lag2',
       'unemployment_rate_lag3', 'CPI_lag1', 'CPI_lag2', 'CPI_lag3',
       'treasury_yield_lag1', 'treasury_yield_lag2', 'treasury_yield_lag3',
       'GDP_growth_lag1', 'GDP_growth_lag2', 'GDP_growth_lag3',
       'SP500_return_lag1', 'SP500_return_lag2', 'SP500_return_lag3',
       'inflation_change_lag1', 'inflation_change_lag2',
       'inflation_change_lag3', 'unemp_change_lag1', 'unemp_change_lag2',
       'unemp_change_lag3', 'treasury_yield_change_lag1',
       'treasury_yield_change_lag2', 'treasury_yield_change_lag3', 'AZN_lag1',
       'AZN_lag2', 'BMY_lag1', 'BMY_lag2', 'JNJ_lag1', 'JNJ_lag2', 'LLY_lag1',
       'LLY_lag2', 'MRK_lag1', 'MRK_lag2', 'NVO_lag1', 'NVO_lag2', 'NVS_lag1',
       

In [8]:
def tick_loss(alpha, returns, var):
    df = pd.DataFrame({'Return': returns, 'VaR': var})
    df['Indicator'] = np.where(df['Return'] < df['VaR'], 1, 0)

    t_loss = 0

    for i in df.index:
        t_loss += (
            alpha * (df['Return'][i] - df['VaR'][i]) * (1 - df['Indicator'][i])
            + (1 - alpha) * (df['VaR'][i] - df['Return'][i]) * df['Indicator'][i]
        )

    return t_loss

In [9]:
def collect_prediction(X_train,y_train,X_test,y_test,estimator,alpha,model_name):
  estimator.fit(X_train,y_train)
  y_pred = estimator.predict(X_test)
  #print( "{model_name} alpha = {alpha:.2f},score = {score:.1f}".format(model_name=model_name, alpha=alpha , score= XGBQuantile.quantile_score(y_test, y_pred, alpha)) )

  return y_pred

In [10]:
dates1m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01', '2022-12-01', '2023-01-01', '2023-02-01', '2023-03-01', '2023-04-01']
dates3m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01', '2022-12-01', '2023-01-01', '2023-02-01']
dates6m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01']
dates9m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01']
dates12m = ['2022-05-01']

In [11]:
to_exclude3 = ['Date']

for covariate in covs:
    to_exclude3.append(covariate)

for t in tickers:
    to_exclude3.append(t)

to_exclude3

['Date',
 'unemployment_rate',
 'CPI',
 'treasury_yield',
 'GDP_growth',
 'SP500_return',
 'inflation_change',
 'unemp_change',
 'treasury_yield_change',
 'AZN',
 'BMY',
 'JNJ',
 'LLY',
 'MRK',
 'NVO',
 'NVS',
 'PFE',
 'ROG']

In [12]:
def rolling_forecast_QR(data, dates, ticker, alpha, columns_to_exclude, horizon):
    t_loss = 0
    dta = data.copy()
    all_var_forecast = []

    for d in dates:
        train_df = dta[dta.Date <= d]
        test_df = dta[dta.Date > d]
        test_df = test_df.reset_index(drop=True)

        X_train = train_df.drop(columns=columns_to_exclude,axis=1)
        y_train = train_df[ticker]

        X_test = test_df.drop(columns=columns_to_exclude, axis=1)
        X_test = X_test.iloc[horizon-1]
        y_test = test_df[ticker][horizon-1]

        quantreg = sm.QuantReg(y_train, X_train)
        quantreg_results = quantreg.fit(q=alpha)

        VaR_forecast = quantreg_results.predict(X_test)
        all_var_forecast.append(VaR_forecast)

        r = y_test
        loss = tick_loss(alpha, r, VaR_forecast)
        t_loss += loss

    combinations = 13 - horizon
    t_loss = t_loss / combinations
    all_var_forecast = np.concatenate(all_var_forecast)

    return t_loss, all_var_forecast

In [13]:
from scipy.stats import norm
from arch import arch_model
import numpy as np

def rolling_forecast_GARCH(data, dates, ticker, alpha, columns_to_exclude, horizon):
    t_loss = 0
    dta = data.copy()
    all_var_forecast = []

    for d in dates:
        train_df = dta[dta.Date <= d]
        test_df = dta[dta.Date > d]
        test_df = test_df.reset_index(drop=True)

        X_train = train_df.drop(columns=columns_to_exclude, axis=1)
        y_train = train_df[ticker]

        X_test = test_df.drop(columns=columns_to_exclude, axis=1)
        X_test = X_test.iloc[horizon-1]
        y_test = test_df[ticker][horizon-1]

        garch_model = arch_model(y_train, vol='Garch', p=1, q=1)
        garch_results = garch_model.fit()

        forecasts = garch_results.forecast(start=0, horizon=1)
        cond_vol_forecast = np.sqrt(forecasts.variance.iloc[-1])
        mean_forecast = forecasts.mean.iloc[-1]

        VaR_forecast = mean_forecast + cond_vol_forecast * norm.ppf(alpha)
        all_var_forecast.append(VaR_forecast)

        loss = tick_loss(alpha, y_test, VaR_forecast)
        t_loss += loss

    combinations = 13 - horizon
    t_loss = t_loss / combinations
    all_var_forecast = np.concatenate(all_var_forecast)

    return t_loss, all_var_forecast

In [14]:
import numpy as np
from xgboost.sklearn import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor

def rolling_forecast_XGB(data, ticker, alpha, exclude, dates, horizon):
    t_loss = 0
    dta = data.copy()
    all_var_forecast = []

    for d in dates:
        split_date = d
        train_df = dta[dta.Date <= split_date]
        train_df = train_df.dropna()
        test_df = dta[dta.Date > split_date]
        test_df = test_df.reset_index(drop=True)

        X_train = train_df.drop(columns=exclude, axis=1)
        X_test = test_df.drop(columns=exclude, axis=1)
        X_test = X_test.iloc[horizon-1]
        X_test_formatted = X_test.values.reshape(1, -1)
        y_train = train_df[ticker]
        y_test = test_df[ticker][horizon-1]

        regressor = GradientBoostingRegressor()
        regressor.set_params(loss='quantile', alpha=0.05)
        y_lower = collect_prediction(X_train, y_train, X_test_formatted, y_test, estimator=regressor, alpha=0.05, model_name="Gradient Boosting")
        tickloss = tick_loss(alpha, y_test, y_lower)
        t_loss += tickloss

        all_var_forecast.append(y_lower)

    combinations = 13 - horizon
    t_loss = t_loss / combinations
    all_var_forecast = np.concatenate(all_var_forecast)

    return t_loss, all_var_forecast

# 1 month

In [15]:
# Computing tick losses and var forecast at horizon=1 for all tickers and all models
tick_losses_all = []
var_forecast_all =[]

for t in tickers:
    # Get tick loss and forecasts for each model
    t_loss_qr, all_var_forecast_qr = rolling_forecast_QR(data, dates1m, t, alpha=0.05, columns_to_exclude=to_exclude3, horizon=1)
    t_loss_garch, all_var_forecast_garch = rolling_forecast_GARCH(data, dates1m, t, alpha=0.05, columns_to_exclude=to_exclude3, horizon=1)
    t_loss_xgboost, all_var_forecast_xgboost = rolling_forecast_XGB(data, t, alpha=0.05, exclude=to_exclude3, dates=dates1m, horizon=1)

    # Combine tick losses into a matrix
    tick_losses = np.column_stack((t_loss_qr, t_loss_garch, t_loss_xgboost))
    tick_losses_all.append(tick_losses)

    # Combine var forecasts into a matrix
    var_forecast = np.column_stack((all_var_forecast_qr,all_var_forecast_garch,all_var_forecast_xgboost))
    var_forecast_all.append(var_forecast)

tick_losses_all = np.array(tick_losses_all)
var_forecast_all = np.array(var_forecast_all)

print("Tick Losses Matrix:")
print(tick_losses_all)
print("VaR Forecasts All:")
print(var_forecast_all)

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


Iteration:      1,   Func. Count:      6,   Neg. LLF: 874.1785677557702
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1200.4721531726964
Iteration:      3,   Func. Count:     19,   Neg. LLF: 870.6852289593519
Iteration:      4,   Func. Count:     26,   Neg. LLF: 867.8399619926613
Iteration:      5,   Func. Count:     32,   Neg. LLF: 867.828481407204
Iteration:      6,   Func. Count:     37,   Neg. LLF: 867.8058039900275
Iteration:      7,   Func. Count:     42,   Neg. LLF: 867.748173849943
Iteration:      8,   Func. Count:     47,   Neg. LLF: 867.7462745676646
Iteration:      9,   Func. Count:     52,   Neg. LLF: 867.7461706287772
Iteration:     10,   Func. Count:     57,   Neg. LLF: 867.746169625292
Iteration:     11,   Func. Count:     61,   Neg. LLF: 867.7461696251956
Optimization terminated successfully    (Exit mode 0)
            Current function value: 867.746169625292
            Iterations: 11
            Function evaluations: 61
            Gradient evaluations: 11
It

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

Iteration:      2,   Func. Count:     13,   Neg. LLF: 900.9392194578554
Iteration:      3,   Func. Count:     19,   Neg. LLF: 898.9513385507807
Iteration:      4,   Func. Count:     26,   Neg. LLF: 895.8637164543424
Iteration:      5,   Func. Count:     31,   Neg. LLF: 895.8604740394894
Iteration:      6,   Func. Count:     36,   Neg. LLF: 895.8453872167669
Iteration:      7,   Func. Count:     41,   Neg. LLF: 895.8115714964167
Iteration:      8,   Func. Count:     46,   Neg. LLF: 895.8111298516814
Iteration:      9,   Func. Count:     51,   Neg. LLF: 895.8111235358674
Iteration:     10,   Func. Count:     55,   Neg. LLF: 895.8111235357464
Optimization terminated successfully    (Exit mode 0)
            Current function value: 895.8111235358674
            Iterations: 10
            Function evaluations: 55
            Gradient evaluations: 10
Iteration:      1,   Func. Count:      6,   Neg. LLF: 902.7968643681711
Iteration:      2,   Func. Count:     12,   Neg. LLF: 938.1897116708617

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

...

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


Iteration:      7,   Func. Count:     43,   Neg. LLF: 1052.4495124313562
Iteration:      8,   Func. Count:     49,   Neg. LLF: 1052.176551037805
Iteration:      9,   Func. Count:     54,   Neg. LLF: 1052.1698786039997
Iteration:     10,   Func. Count:     59,   Neg. LLF: 1052.169349871926
Iteration:     11,   Func. Count:     64,   Neg. LLF: 1052.1692421264295
Iteration:     12,   Func. Count:     69,   Neg. LLF: 1052.1692313108813
Iteration:     13,   Func. Count:     73,   Neg. LLF: 1052.1692313133951
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1052.1692313108813
            Iterations: 13
            Function evaluations: 73
            Gradient evaluations: 13
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1174.1994644695042
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1135.118426737945
Iteration:      3,   Func. Count:     20,   Neg. LLF: 1056.4270033692883
Iteration:      4,   Func. Count:     25,   Neg. LLF: 1056.3051

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature nam

Tick Losses Matrix:
[[[0.89672138 0.75102131 0.72911825]]

 [[0.71467371 0.48696721 0.49212654]]

 [[2.31440119 0.46835748 0.60720606]]

 [[1.36682505 0.69379176 0.67465481]]

 [[0.65496638 0.58647176 0.59382063]]

 [[0.7442595  0.66693109 0.6294229 ]]

 [[1.49964732 0.45892824 0.51931541]]

 [[2.31069231 0.77356387 0.88329887]]

 [[4.4995135  3.98913715 4.0348687 ]]]
VaR Forecasts All:
[[[ -9.83728109  -9.15195024  -8.51924071]
  [ -2.06133914  -8.89037173  -7.34639808]
  [ -3.47685705  -8.66274921  -8.12191188]
  [ -9.30813624  -9.12655521  -8.31819168]
  [-12.5588184  -10.64292001 -10.37844619]
  [ -7.23167789 -10.35228813  -7.91603678]
  [ -7.97638183 -12.05033945  -7.87657495]
  [-15.34285013 -10.84558587  -8.1672183 ]
  [-14.46254905 -10.29198289  -7.70781324]
  [ -1.77012818  -9.69677902  -7.70517943]
  [ -7.88142787  -9.81479006  -7.72481183]
  [-19.04914778  -9.55508869  -7.87557201]]

 [[ -9.43470816  -9.73720522 -10.31195861]
  [-10.86444788  -9.61872766  -9.98187436]
  [ -8

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(


In [16]:
tick_losses_all = np.array(tick_losses_all)
tick_losses_all_matrix = np.reshape(tick_losses_all, (9, 3))
tick_losses_all_matrix

array([[0.89672138, 0.75102131, 0.72911825],
       [0.71467371, 0.48696721, 0.49212654],
       [2.31440119, 0.46835748, 0.60720606],
       [1.36682505, 0.69379176, 0.67465481],
       [0.65496638, 0.58647176, 0.59382063],
       [0.7442595 , 0.66693109, 0.6294229 ],
       [1.49964732, 0.45892824, 0.51931541],
       [2.31069231, 0.77356387, 0.88329887],
       [4.4995135 , 3.98913715, 4.0348687 ]])

In [17]:
# Compute weights for each particular ticker with higher weight to lower tick loss models
weights_per_ticker = []
for tick_losses in tick_losses_all_matrix:
    inverted_tick_losses = 1 / tick_losses
    total_inverted_loss = np.sum(inverted_tick_losses)
    weights = inverted_tick_losses / total_inverted_loss
    weights_per_ticker.append(weights)

# Print the weights for each particular ticker
for ticker, weights in enumerate(weights_per_ticker):
    print(f"Weights for Ticker {ticker+1}: {weights}")

Weights for Ticker 1: [0.29206686 0.34872858 0.35920456]
Weights for Ticker 2: [0.25511397 0.3744056  0.37048043]
Weights for Ticker 3: [0.10253166 0.50666297 0.39080537]
Weights for Ticker 4: [0.20015846 0.39432811 0.40551344]
Weights for Ticker 5: [0.31058205 0.34685524 0.3425627 ]
Weights for Ticker 6: [0.30317784 0.33833029 0.35849187]
Weights for Ticker 7: [0.13975351 0.45667483 0.40357166]
Weights for Ticker 8: [0.15144508 0.45237761 0.39617732]
Weights for Ticker 9: [0.30834706 0.34779745 0.34385549]


In [18]:
weights_array = np.array(weights_per_ticker)
weights_array

array([[0.29206686, 0.34872858, 0.35920456],
       [0.25511397, 0.3744056 , 0.37048043],
       [0.10253166, 0.50666297, 0.39080537],
       [0.20015846, 0.39432811, 0.40551344],
       [0.31058205, 0.34685524, 0.3425627 ],
       [0.30317784, 0.33833029, 0.35849187],
       [0.13975351, 0.45667483, 0.40357166],
       [0.15144508, 0.45237761, 0.39617732],
       [0.30834706, 0.34779745, 0.34385549]])

In [19]:
# Multiply each VaR forecast matrix by the corresponding weights
weighted_var_forecasts = []
row_sums_per_ticker = []
for ticker in range(len(var_forecast_all)):
    weights = weights_array[ticker]
    var_matrix = var_forecast_all[ticker]
    weighted_var_matrix = var_matrix * np.transpose(weights)
    weighted_var_forecasts.append(weighted_var_matrix)
    row_sums = np.sum(weighted_var_matrix, axis=1)
    row_sums_per_ticker.append(row_sums)

# Print the weighted VaR forecast matrices and their row sums (weithed ensemble predictions)
for ticker, (weighted_var_matrix, row_sums) in enumerate(zip(weighted_var_forecasts, row_sums_per_ticker)):
    print(f"Weighted VaR Forecast Matrix for Ticker {ticker+1}:")
    print(weighted_var_matrix)
    print(f"Row sums for Ticker {ticker+1}:")
    print(row_sums)

Weighted VaR Forecast Matrix for Ticker 1:
[[-2.87314378 -3.19154665 -3.06015009]
 [-0.60204885 -3.10032674 -2.63885967]
 [-1.01547471 -3.02094826 -2.91742776]
 [-2.71859811 -3.18269067 -2.98793237]
 [-3.66801463 -3.71149042 -3.72798518]
 [-2.11213344 -3.61013878 -2.84347649]
 [-2.32963678 -4.20229781 -2.82930162]
 [-4.48113803 -3.7821658  -2.93370204]
 [-4.22403126 -3.58910862 -2.76868165]
 [-0.51699578 -3.38154402 -2.76773557]
 [-2.30190388 -3.42269784 -2.77478762]
 [-5.56362474 -3.33213255 -2.82894136]]
Row sums for Ticker 1:
[ -9.12484052  -6.34123526  -6.95385074  -8.88922115 -11.10749023
  -8.56574872  -9.36123622 -11.19700588 -10.58182153  -6.66627536
  -8.49938933 -11.72469865]
Weighted VaR Forecast Matrix for Ticker 2:
[[-2.40692589 -3.64566417 -3.82037881]
 [-2.77167247 -3.60130551 -3.69808906]
 [-2.17648231 -3.70573132 -4.07381354]
 [ 0.38236276 -4.1887248  -3.75512292]
 [-3.86404176 -3.92016767 -4.06596085]
 [-4.13724557 -4.09073543 -4.06377353]
 [-4.78092737 -3.82413507 -4

In [20]:
## Print the weighted VaR forecast matrices and their row sums (weithed ensemble predictions)
for t, (weighted_var_matrix, row_sums) in enumerate(zip(weighted_var_forecasts, row_sums_per_ticker)):
    print(f"Ensemble forecasts for Ticker {t+1}:")
    print(row_sums)

Ensemble forecasts for Ticker 1:
[ -9.12484052  -6.34123526  -6.95385074  -8.88922115 -11.10749023
  -8.56574872  -9.36123622 -11.19700588 -10.58182153  -6.66627536
  -8.49938933 -11.72469865]
Ensemble forecasts for Ticker 2:
[ -9.87296887 -10.07106704  -9.95602718  -7.56148496 -11.85017027
 -12.29175453 -12.66090443 -11.77038953  -8.48907525 -11.39884723
 -12.89812572  -9.59803159]
Ensemble forecasts for Ticker 3:
[-5.43007128 -5.33565354 -4.31351629 -6.42631917 -6.42600965 -7.61954219
 -5.05405712 -6.79964155 -6.94759866 -7.39533867 -7.23179261 -7.35026733]
Ensemble forecasts for Ticker 4:
[ -9.2281157   -9.70864865  -7.35239836  -8.14599628  -8.71886427
  -9.34981011  -8.54384007  -8.45688226  -9.1097952   -7.84370584
  -9.39630418 -10.35956247]
Ensemble forecasts for Ticker 5:
[ -8.29264028 -13.67972724 -13.53403874  -6.97841011  -9.04509832
 -11.21552031 -13.735999    -9.84099542 -10.57150004  -8.43764441
  -7.7026637   -9.69747033]
Ensemble forecasts for Ticker 6:
[ -8.67238643 -

In [21]:
for t in range(len(row_sums_per_ticker)):
    return_ticker = row_sums_per_ticker[t]
    print(f"Ensemble forecasts for Ticker {t+1}:",return_ticker)

Ensemble forecasts for Ticker 1: [ -9.12484052  -6.34123526  -6.95385074  -8.88922115 -11.10749023
  -8.56574872  -9.36123622 -11.19700588 -10.58182153  -6.66627536
  -8.49938933 -11.72469865]
Ensemble forecasts for Ticker 2: [ -9.87296887 -10.07106704  -9.95602718  -7.56148496 -11.85017027
 -12.29175453 -12.66090443 -11.77038953  -8.48907525 -11.39884723
 -12.89812572  -9.59803159]
Ensemble forecasts for Ticker 3: [-5.43007128 -5.33565354 -4.31351629 -6.42631917 -6.42600965 -7.61954219
 -5.05405712 -6.79964155 -6.94759866 -7.39533867 -7.23179261 -7.35026733]
Ensemble forecasts for Ticker 4: [ -9.2281157   -9.70864865  -7.35239836  -8.14599628  -8.71886427
  -9.34981011  -8.54384007  -8.45688226  -9.1097952   -7.84370584
  -9.39630418 -10.35956247]
Ensemble forecasts for Ticker 5: [ -8.29264028 -13.67972724 -13.53403874  -6.97841011  -9.04509832
 -11.21552031 -13.735999    -9.84099542 -10.57150004  -8.43764441
  -7.7026637   -9.69747033]
Ensemble forecasts for Ticker 6: [ -8.67238643 -

In [22]:
return_ticker.flatten()

array([-10.46306228, -23.5647436 , -24.68569826, -16.74372029,
       -17.72569165, -16.57939537, -29.39707391, -10.81561119,
       -21.05819745, -23.55095323, -18.23566173, -13.51834656])

In [23]:
Var_list = []
for t in range(len(row_sums_per_ticker)):
    return_ticker = row_sums_per_ticker[t]
    Var_list.append(return_ticker)

In [24]:
ret_list = []
for t in tickers:
    ret_list.append(data[t].tail(12))

In [25]:
tick_loss_list=[]
for i in range(len(Var_list)):
    tick_loss_ensemble = tick_loss(alpha=0.05, returns= ret_list[i], var=Var_list[i])
    tick_loss_list.append(tick_loss_ensemble)

In [26]:
tick_loss_list_1m=[]
for i in range(len(tick_loss_list)):
    tick_loss_list_1m.append(tick_loss_list[i]/12)

tick_loss_list_1m

[0.7288467006282748,
 0.4899556394139737,
 0.6012000305354587,
 0.7408906546520928,
 0.6102624072734912,
 0.6583464792358977,
 0.5604525335513221,
 0.9786824270013966,
 3.924907412339454]

# 3 months

In [27]:
# Computing tick losses and var forecast at horizon=1 for all tickers and all models
tick_losses_all_3m = []
var_forecast_all_3m =[]

for t in tickers:
    # Get tick loss and forecasts for each model
    t_loss_qr, all_var_forecast_qr = rolling_forecast_QR(data, dates3m, t, alpha=0.05, columns_to_exclude=to_exclude3, horizon=3)
    t_loss_garch, all_var_forecast_garch = rolling_forecast_GARCH(data, dates3m, t, alpha=0.05, columns_to_exclude=to_exclude3, horizon=3)
    t_loss_xgboost, all_var_forecast_xgboost = rolling_forecast_XGB(data, t, alpha=0.05, exclude=to_exclude3, dates=dates3m, horizon=3)

    # Combine tick losses into a matrix
    tick_losses = np.column_stack((t_loss_qr, t_loss_garch, t_loss_xgboost))
    tick_losses_all_3m.append(tick_losses)

    # Combine var forecasts into a matrix
    var_forecast = np.column_stack((all_var_forecast_qr,all_var_forecast_garch,all_var_forecast_xgboost))
    var_forecast_all_3m.append(var_forecast)

tick_losses_all_3m = np.array(tick_losses_all_3m)
var_forecast_all_3m = np.array(var_forecast_all_3m)

print("Tick Losses Matrix:")
print(tick_losses_all_3m)
print("VaR Forecasts All:")
print(var_forecast_all_3m)

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


Iteration:      1,   Func. Count:      6,   Neg. LLF: 874.1785677557702
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1200.4721531726964
Iteration:      3,   Func. Count:     19,   Neg. LLF: 870.6852289593519
Iteration:      4,   Func. Count:     26,   Neg. LLF: 867.8399619926613
Iteration:      5,   Func. Count:     32,   Neg. LLF: 867.828481407204
Iteration:      6,   Func. Count:     37,   Neg. LLF: 867.8058039900275
Iteration:      7,   Func. Count:     42,   Neg. LLF: 867.748173849943
Iteration:      8,   Func. Count:     47,   Neg. LLF: 867.7462745676646
Iteration:      9,   Func. Count:     52,   Neg. LLF: 867.7461706287772
Iteration:     10,   Func. Count:     57,   Neg. LLF: 867.746169625292
Iteration:     11,   Func. Count:     61,   Neg. LLF: 867.7461696251956
Optimization terminated successfully    (Exit mode 0)
            Current function value: 867.746169625292
            Iterations: 11
            Function evaluations: 61
            Gradient evaluations: 11
It

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

Iteration:      2,   Func. Count:     13,   Neg. LLF: 886.8361595084345
Iteration:      3,   Func. Count:     19,   Neg. LLF: 884.5076379612233
Iteration:      4,   Func. Count:     26,   Neg. LLF: 881.4156102171411
Iteration:      5,   Func. Count:     31,   Neg. LLF: 881.4116765176805
Iteration:      6,   Func. Count:     36,   Neg. LLF: 881.3924911968852
Iteration:      7,   Func. Count:     41,   Neg. LLF: 881.3541270434575
Iteration:      8,   Func. Count:     46,   Neg. LLF: 881.353888375146
Iteration:      9,   Func. Count:     51,   Neg. LLF: 881.3538747215401
Iteration:     10,   Func. Count:     55,   Neg. LLF: 881.3538747213574
Optimization terminated successfully    (Exit mode 0)
            Current function value: 881.3538747215401
            Iterations: 10
            Function evaluations: 55
            Gradient evaluations: 10
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1227.934884202285
Iteration:      2,   Func. Count:     13,   Neg. LLF: 889.2892608331276


/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

...

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

Iteration:     11,   Func. Count:     66,   Neg. LLF: 1044.3919030758052
Iteration:     12,   Func. Count:     71,   Neg. LLF: 1044.3918989609401
Iteration:     13,   Func. Count:     76,   Neg. LLF: 1044.3918984136199
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1044.3918984136199
            Iterations: 13
            Function evaluations: 76
            Gradient evaluations: 13
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1165.6506883972468
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1108.8767328283766
Iteration:      3,   Func. Count:     20,   Neg. LLF: 1049.122615254116
Iteration:      4,   Func. Count:     25,   Neg. LLF: 1053.8626624317908
Iteration:      5,   Func. Count:     31,   Neg. LLF: 1056.682904891818
Iteration:      6,   Func. Count:     37,   Neg. LLF: 1051.8795712975527
Iteration:      7,   Func. Count:     43,   Neg. LLF: 1050.234448466702
Iteration:      8,   Func. Count:     49,   Neg. LLF: 1049.4115

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature nam

Tick Losses Matrix:
[[[1.18373133 0.82986734 0.89844034]]

 [[0.76002999 0.47591979 0.49421014]]

 [[1.90616054 0.45785688 0.3879952 ]]

 [[1.55390659 0.68951431 0.68244833]]

 [[0.66455056 0.61807923 0.63069791]]

 [[0.71854137 0.65185394 0.63241683]]

 [[1.59359328 0.48209179 0.50462969]]

 [[2.711749   0.74724328 0.99864134]]

 [[4.9234891  4.52418838 4.62100043]]]
VaR Forecasts All:
[[[ -0.72719114  -9.15195024  -8.75467099]
  [-10.29957562  -8.89037173  -7.35262271]
  [-11.69836103  -8.66274921  -7.98150989]
  [ -7.54087514  -9.12655521  -7.84249292]
  [ -5.94773453 -10.64292001  -9.95346557]
  [-15.86439512 -10.35228813  -8.63690836]
  [-14.74726295 -12.05033945  -7.90016293]
  [ -2.15376481 -10.84558587  -7.71340913]
  [ -8.23424843 -10.29198289  -7.6378557 ]
  [-19.7372253   -9.69677902  -8.89804276]]

 [[ -8.43641753  -9.73720522 -11.03553387]
  [  1.7290061   -9.61872766 -10.37707216]
  [-15.14294797  -9.89763858 -10.32777057]
  [-13.4810379  -11.18766598 -10.98202974]
  [-18

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(


In [28]:
tick_losses_all_3m = np.array(tick_losses_all_3m)
tick_losses_all_3m_matrix = np.reshape(tick_losses_all_3m, (9, 3))
tick_losses_all_3m_matrix

array([[1.18373133, 0.82986734, 0.89844034],
       [0.76002999, 0.47591979, 0.49421014],
       [1.90616054, 0.45785688, 0.3879952 ],
       [1.55390659, 0.68951431, 0.68244833],
       [0.66455056, 0.61807923, 0.63069791],
       [0.71854137, 0.65185394, 0.63241683],
       [1.59359328, 0.48209179, 0.50462969],
       [2.711749  , 0.74724328, 0.99864134],
       [4.9234891 , 4.52418838, 4.62100043]])

In [29]:
# Compute weights for each particular ticker with higher weight to lower tick loss models
weights_per_ticker_3m = []
for tick_losses in tick_losses_all_3m_matrix:
    inverted_tick_losses = 1 / tick_losses
    total_inverted_loss = np.sum(inverted_tick_losses)
    weights = inverted_tick_losses / total_inverted_loss
    weights_per_ticker_3m.append(weights)

In [30]:
# Print the weights for each particular ticker
for ticker, weights in enumerate(weights_per_ticker_3m):
    print(f"Weights for Ticker {ticker+1}: {weights}")

Weights for Ticker 1: [0.26709754 0.38099069 0.35191177]
Weights for Ticker 2: [0.2418474  0.38622322 0.37192938]
Weights for Ticker 3: [0.09924505 0.41317933 0.48757562]
Weights for Ticker 4: [0.18081275 0.40748411 0.41170314]
Weights for Ticker 5: [0.31960506 0.34363511 0.33675983]
Weights for Ticker 6: [0.30878613 0.34037626 0.35083761]
Weights for Ticker 7: [0.13398492 0.44289795 0.42311713]
Weights for Ticker 8: [0.13615737 0.49411568 0.36972695]
Weights for Ticker 9: [0.31708599 0.34507171 0.3378423 ]


In [31]:
weights_array_3m = np.array(weights_per_ticker_3m)

In [32]:
# Multiply each VaR forecast matrix by the corresponding weights
weighted_var_forecasts_3m = []
row_sums_per_ticker_3m = []
for ticker in range(len(var_forecast_all_3m)):
    weights = weights_array_3m[ticker]
    var_matrix = var_forecast_all_3m[ticker]
    weighted_var_matrix = var_matrix * np.transpose(weights)
    weighted_var_forecasts_3m.append(weighted_var_matrix)
    row_sums_3m = np.sum(weighted_var_matrix, axis=1)
    row_sums_per_ticker_3m.append(row_sums_3m)

# Print the weighted VaR forecast matrices and their row sums (weithed ensemble predictions)
for ticker, (weighted_var_matrix, row_sums_3m) in enumerate(zip(weighted_var_forecasts_3m, row_sums_per_ticker_3m)):
    print(f"Weighted VaR Forecast Matrix for Ticker {ticker+1}:")
    print(weighted_var_matrix)
    print(f"Row sums for Ticker {ticker+1}:")
    print(row_sums_3m)

Weighted VaR Forecast Matrix for Ticker 1:
[[-0.19423097 -3.48680782 -3.08087176]
 [-2.75099134 -3.38714884 -2.58747447]
 [-3.12460349 -3.30042678 -2.80878727]
 [-2.01414922 -3.47713255 -2.75986556]
 [-1.58862528 -4.05485342 -3.50274168]
 [-4.23734096 -3.94412538 -3.0394297 ]
 [-3.9389577  -4.59106712 -2.78016032]
 [-0.57526529 -4.13206722 -2.71443945]
 [-2.19934752 -3.92114964 -2.68785131]
 [-5.27176438 -3.69438251 -3.13132597]]
Row sums for Ticker 1:
[ -6.76191054  -8.72561465  -9.23381753  -8.25114732  -9.14622038
 -11.22089603 -11.31018513  -7.42177196  -8.80834848 -12.09747286]
Weighted VaR Forecast Matrix for Ticker 2:
[[-2.04032561 -3.76073476 -4.10443932]
 [ 0.41815562 -3.71497597 -3.85953806]
 [-3.66228253 -3.82269785 -3.84120135]
 [-3.2603539  -4.32093638 -4.08453956]
 [-4.56130236 -4.0439026  -3.92507727]
 [-3.35350996 -4.2198541  -4.07966706]
 [-0.45257262 -3.94483886 -4.0717045 ]
 [-2.84196965 -4.53339244 -4.09572782]
 [-4.51691464 -4.02918325 -4.09222229]
 [-1.91823415 -3

In [33]:
for t in range(len(row_sums_per_ticker_3m)):
    return_ticker_3m = row_sums_per_ticker_3m[t]
    print(f"Ensemble forecasts for Ticker {t+1}:",return_ticker_3m)

Ensemble forecasts for Ticker 1: [ -6.76191054  -8.72561465  -9.23381753  -8.25114732  -9.14622038
 -11.22089603 -11.31018513  -7.42177196  -8.80834848 -12.09747286]
Ensemble forecasts for Ticker 2: [ -9.90549968  -7.15635841 -11.32618172 -11.66582985 -12.53028224
 -11.65303112  -8.46911598 -11.47108992 -12.63832018 -10.00017813]
Ensemble forecasts for Ticker 3: [-5.08432358 -5.84894831 -5.9811368  -7.60334865 -5.37006263 -7.81275904
 -7.09687421 -6.6217519  -6.85781081 -7.30031141]
Ensemble forecasts for Ticker 4: [-8.1518671  -9.02498557 -8.05182121 -9.28898312 -8.83324995 -9.03393644
 -9.44263332 -7.63976886 -8.98732429 -9.74816088]
Ensemble forecasts for Ticker 5: [-13.84762019  -7.09128406  -8.7587631  -10.73039388 -13.33430547
 -11.48965527 -10.19890687  -8.42688337  -7.82309289  -9.86431332]
Ensemble forecasts for Ticker 6: [ -8.89773263 -10.04382504  -8.99761681  -6.92146305  -7.48014408
 -14.38629461  -9.14318748  -7.45184417  -7.53165702 -12.32010058]
Ensemble forecasts for T

In [34]:
return_ticker_3m.flatten()

array([-26.60129996, -17.66922154, -16.60139698, -16.76831048,
       -11.3068583 , -11.91996872, -33.30772649, -22.04536652,
       -18.52477156, -12.96296469])

In [35]:
Var_list_3m = []
for t in range(len(row_sums_per_ticker_3m)):
    return_ticker_3m = row_sums_per_ticker_3m[t]
    Var_list_3m.append(return_ticker_3m)

ret_list_3m = []
for t in tickers:
    ret_list_3m.append(data[t].tail(10))

tick_loss_list_3=[]
for i in range(len(Var_list_3m)):
    tick_loss_ensemble = tick_loss(alpha=0.05, returns= ret_list_3m[i], var=Var_list_3m[i])
    tick_loss_list_3.append(tick_loss_ensemble)

tick_loss_list_3m=[]
for i in range(len(tick_loss_list_3)):
    tick_loss_list_3m.append(tick_loss_list_3[i]/10)

tick_loss_list_3m

[0.8126723628990543,
 0.48724145165590915,
 0.5324936724048341,
 0.6584070873266737,
 0.6371811666816838,
 0.6511052887483438,
 0.5407314527854148,
 0.927031551842387,
 4.490385269586563]

# 6 months

In [36]:
# Computing tick losses and var forecast at horizon=1 for all tickers and all models
tick_losses_all_6m = []
var_forecast_all_6m =[]

for t in tickers:
    # Get tick loss and forecasts for each model
    t_loss_qr, all_var_forecast_qr = rolling_forecast_QR(data, dates6m, t, alpha=0.05, columns_to_exclude=to_exclude3, horizon=6)
    t_loss_garch, all_var_forecast_garch = rolling_forecast_GARCH(data, dates6m, t, alpha=0.05, columns_to_exclude=to_exclude3, horizon=6)
    t_loss_xgboost, all_var_forecast_xgboost = rolling_forecast_XGB(data, t, alpha=0.05, exclude=to_exclude3, dates=dates6m, horizon=6)

    # Combine tick losses into a matrix
    tick_losses = np.column_stack((t_loss_qr, t_loss_garch, t_loss_xgboost))
    tick_losses_all_6m.append(tick_losses)

    # Combine var forecasts into a matrix
    var_forecast = np.column_stack((all_var_forecast_qr,all_var_forecast_garch,all_var_forecast_xgboost))
    var_forecast_all_6m.append(var_forecast)

tick_losses_all_6m = np.array(tick_losses_all_6m)
var_forecast_all_6m = np.array(var_forecast_all_6m)

print("Tick Losses Matrix:")
print(tick_losses_all_3m)
print("VaR Forecasts All:")
print(var_forecast_all_3m)

Iteration:      1,   Func. Count:      6,   Neg. LLF: 874.1785677557702
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1200.4721531726964
Iteration:      3,   Func. Count:     19,   Neg. LLF: 870.6852289593519
Iteration:      4,   Func. Count:     26,   Neg. LLF: 867.8399619926613
Iteration:      5,   Func. Count:     32,   Neg. LLF: 867.828481407204
Iteration:      6,   Func. Count:     37,   Neg. LLF: 867.8058039900275
Iteration:      7,   Func. Count:     42,   Neg. LLF: 867.748173849943
Iteration:      8,   Func. Count:     47,   Neg. LLF: 867.7462745676646
Iteration:      9,   Func. Count:     52,   Neg. LLF: 867.7461706287772
Iteration:     10,   Func. Count:     57,   Neg. LLF: 867.746169625292
Iteration:     11,   Func. Count:     61,   Neg. LLF: 867.7461696251956
Optimization terminated successfully    (Exit mode 0)
            Current function value: 867.746169625292
            Iterations: 11
            Function evaluations: 61
            Gradient evaluations: 11
It

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

Iteration:      1,   Func. Count:      6,   Neg. LLF: 904.2346809763922
Iteration:      2,   Func. Count:     13,   Neg. LLF: 909.8940809007032
Iteration:      3,   Func. Count:     19,   Neg. LLF: 901.9676011216195
Iteration:      4,   Func. Count:     26,   Neg. LLF: 885.2200651421106
Iteration:      5,   Func. Count:     31,   Neg. LLF: 885.2195442810739
Iteration:      6,   Func. Count:     36,   Neg. LLF: 885.2195104425602
Iteration:      7,   Func. Count:     41,   Neg. LLF: 885.2193138255832
Iteration:      8,   Func. Count:     46,   Neg. LLF: 885.2183522300605
Iteration:      9,   Func. Count:     51,   Neg. LLF: 885.2147616595502
Iteration:     10,   Func. Count:     56,   Neg. LLF: 885.2128950597746
Iteration:     11,   Func. Count:     61,   Neg. LLF: 885.2127609855501
Iteration:     12,   Func. Count:     66,   Neg. LLF: 885.2127585731427
Iteration:     13,   Func. Count:     70,   Neg. LLF: 885.2127585730582
Optimization terminated successfully    (Exit mode 0)
          

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

Iteration:      1,   Func. Count:      6,   Neg. LLF: 3737.673074947868
Iteration:      2,   Func. Count:     14,   Neg. LLF: 1059.3458407887865
Iteration:      3,   Func. Count:     20,   Neg. LLF: 990.3442313922687
Iteration:      4,   Func. Count:     26,   Neg. LLF: 774.7400448569347
Iteration:      5,   Func. Count:     32,   Neg. LLF: 779.060699049972
Iteration:      6,   Func. Count:     38,   Neg. LLF: 771.3832501664328
Iteration:      7,   Func. Count:     44,   Neg. LLF: 769.7415752103974
Iteration:      8,   Func. Count:     50,   Neg. LLF: 765.6335026311029
Iteration:      9,   Func. Count:     55,   Neg. LLF: 765.6297294381544
Iteration:     10,   Func. Count:     60,   Neg. LLF: 765.6289482309151
Iteration:     11,   Func. Count:     65,   Neg. LLF: 765.628625130922
Iteration:     12,   Func. Count:     70,   Neg. LLF: 765.6283320636917
Iteration:     13,   Func. Count:     75,   Neg. LLF: 765.6283000831261
Iteration:     14,   Func. Count:     80,   Neg. LLF: 765.6282984

...

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fit

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1210.4827289658347
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1011.3744844764094
Iteration:      3,   Func. Count:     19,   Neg. LLF: 1009.5684780724663
Iteration:      4,   Func. Count:     24,   Neg. LLF: 1009.5683778425991
Iteration:      5,   Func. Count:     29,   Neg. LLF: 1009.5682735267333
Iteration:      6,   Func. Count:     34,   Neg. LLF: 1009.5680105828636
Iteration:      7,   Func. Count:     39,   Neg. LLF: 1009.5663903221539
Iteration:      8,   Func. Count:     44,   Neg. LLF: 1009.5628278473197
Iteration:      9,   Func. Count:     49,   Neg. LLF: 1009.562807759047
Iteration:     10,   Func. Count:     53,   Neg. LLF: 1009.5628077585338
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1009.562807759047
            Iterations: 10
            Function evaluations: 53
            Gradient evaluations: 10
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1215.021

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

Tick Losses Matrix:
[[[1.18373133 0.82986734 0.89844034]]

 [[0.76002999 0.47591979 0.49421014]]

 [[1.90616054 0.45785688 0.3879952 ]]

 [[1.55390659 0.68951431 0.68244833]]

 [[0.66455056 0.61807923 0.63069791]]

 [[0.71854137 0.65185394 0.63241683]]

 [[1.59359328 0.48209179 0.50462969]]

 [[2.711749   0.74724328 0.99864134]]

 [[4.9234891  4.52418838 4.62100043]]]
VaR Forecasts All:
[[[ -0.72719114  -9.15195024  -8.75467099]
  [-10.29957562  -8.89037173  -7.35262271]
  [-11.69836103  -8.66274921  -7.98150989]
  [ -7.54087514  -9.12655521  -7.84249292]
  [ -5.94773453 -10.64292001  -9.95346557]
  [-15.86439512 -10.35228813  -8.63690836]
  [-14.74726295 -12.05033945  -7.90016293]
  [ -2.15376481 -10.84558587  -7.71340913]
  [ -8.23424843 -10.29198289  -7.6378557 ]
  [-19.7372253   -9.69677902  -8.89804276]]

 [[ -8.43641753  -9.73720522 -11.03553387]
  [  1.7290061   -9.61872766 -10.37707216]
  [-15.14294797  -9.89763858 -10.32777057]
  [-13.4810379  -11.18766598 -10.98202974]
  [-18

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(


In [37]:
tick_losses_all_6m = np.array(tick_losses_all_6m)
tick_losses_all_6m_matrix = np.reshape(tick_losses_all_6m, (9, 3))
tick_losses_all_6m_matrix

array([[0.72207601, 0.66900835, 0.60245663],
       [0.83108333, 0.51452167, 0.48255404],
       [0.51141465, 0.41118612, 0.54444965],
       [0.50610708, 0.68809424, 0.72096019],
       [0.61992755, 0.56213117, 0.56942527],
       [0.82652696, 0.76076521, 0.70531213],
       [1.0924455 , 0.55408456, 0.61652547],
       [0.55567616, 0.78590746, 1.05382987],
       [6.23886966, 5.99535045, 6.27733903]])

In [38]:

# Compute weights for each particular ticker with higher weight to lower tick loss models
weights_per_ticker_6m = []
for tick_losses in tick_losses_all_6m_matrix:
    inverted_tick_losses = 1 / tick_losses
    total_inverted_loss = np.sum(inverted_tick_losses)
    weights = inverted_tick_losses / total_inverted_loss
    weights_per_ticker_6m.append(weights)

In [39]:
# Print the weights for each particular ticker
for ticker, weights in enumerate(weights_per_ticker_6m):
    print(f"Weights for Ticker {ticker+1}: {weights}")

Weights for Ticker 1: [0.30507565 0.32927512 0.36564923]
Weights for Ticker 2: [0.23054681 0.37239173 0.39706146]
Weights for Ticker 3: [0.31416125 0.39073953 0.29509922]
Weights for Ticker 4: [0.41025467 0.30175052 0.28799481]
Weights for Ticker 5: [0.31333166 0.34554733 0.34112101]
Weights for Ticker 6: [0.30690838 0.33343802 0.3596536 ]
Weights for Ticker 7: [0.21081207 0.4156418  0.37354613]
Weights for Ticker 8: [0.44755896 0.31644673 0.2359943 ]
Weights for Ticker 9: [0.3295447  0.34293015 0.32752515]


In [40]:
weights_array_6m = np.array(weights_per_ticker_6m)

In [41]:
# Multiply each VaR forecast matrix by the corresponding weights
weighted_var_forecasts_6m = []
row_sums_per_ticker_6m = []
for ticker in range(len(var_forecast_all_6m)):
    weights = weights_array_6m[ticker]
    var_matrix = var_forecast_all_6m[ticker]
    weighted_var_matrix = var_matrix * np.transpose(weights)
    weighted_var_forecasts_6m.append(weighted_var_matrix)
    row_sums_6m = np.sum(weighted_var_matrix, axis=1)
    row_sums_per_ticker_6m.append(row_sums_6m)

# Print the weighted VaR forecast matrices and their row sums (weithed ensemble predictions)
for ticker, (weighted_var_matrix, row_sums_6m) in enumerate(zip(weighted_var_forecasts_6m, row_sums_per_ticker_6m)):
    print(f"Weighted VaR Forecast Matrix for Ticker {ticker+1}:")
    print(weighted_var_matrix)
    print(f"Row sums for Ticker {ticker+1}:")
    print(row_sums_6m)

Weighted VaR Forecast Matrix for Ticker 1:
[[-1.91054871 -3.01350951 -3.11505384]
 [-1.49954137 -2.92737821 -2.91720423]
 [-5.17287396 -2.85242778 -3.05861461]
 [-4.52762866 -3.00514756 -2.88167797]
 [-0.57432416 -3.50444876 -3.70819031]
 [-2.71561513 -3.40875091 -2.98158289]
 [-6.87876732 -3.96787696 -3.11571994]]
Row sums for Ticker 1:
[ -8.03911206  -7.34412382 -11.08391635 -10.41445419  -7.78696322
  -9.10594892 -13.96236422]
Weighted VaR Forecast Matrix for Ticker 2:
[[-3.16237211 -3.62605465 -4.38178521]
 [-4.34032054 -3.58193459 -3.96371408]
 [-3.07460899 -3.68579871 -4.11070201]
 [-0.46211992 -4.16619424 -4.36054079]
 [-2.61007185 -3.89908163 -4.1892462 ]
 [-4.40968323 -4.06873193 -4.35533904]
 [-1.54740947 -3.80356558 -4.36024841]]
Row sums for Ticker 2:
[-11.17021198 -11.88596921 -10.87110971  -8.98885495 -10.69839969
 -12.83375421  -9.71122346]
Weighted VaR Forecast Matrix for Ticker 3:
[[-2.32378251 -2.6125334  -1.96089222]
 [-1.83176263 -2.55962389 -1.61413294]
 [-5.904219

In [42]:
for t in range(len(row_sums_per_ticker_6m)):
    return_ticker_6m = row_sums_per_ticker_6m[t]
    print(f"Ensemble forecasts for Ticker {t+1}:",return_ticker_6m)

Ensemble forecasts for Ticker 1: [ -8.03911206  -7.34412382 -11.08391635 -10.41445419  -7.78696322
  -9.10594892 -13.96236422]
Ensemble forecasts for Ticker 2: [-11.17021198 -11.88596921 -10.87110971  -8.98885495 -10.69839969
 -12.83375421  -9.71122346]
Ensemble forecasts for Ticker 3: [ -6.89720812  -6.00551947 -10.04589968  -7.37974902  -8.42213728
  -8.97168265  -9.17140139]
Ensemble forecasts for Ticker 4: [ -9.39335335  -7.72326208  -9.54581203  -9.65134637  -5.85456358
  -8.97127854 -10.03362825]
Ensemble forecasts for Ticker 5: [-11.4117574  -14.29929519  -9.2906649  -10.13808121  -7.98728616
  -7.74904017  -9.72813105]
Ensemble forecasts for Ticker 6: [ -6.6173055   -7.12760506 -14.40650075  -8.7959995   -7.18101731
  -7.3438726  -13.09229602]
Ensemble forecasts for Ticker 7: [-6.39820883 -5.75373929 -9.19177018 -6.26577137 -6.84432672 -7.34312009
 -9.30358485]
Ensemble forecasts for Ticker 8: [ -8.72068169  -8.72026137 -11.39280546 -11.77372241  -9.24739472
  -9.75201233 -13.0

In [43]:
return_ticker_6m.flatten()

array([-17.51940912, -14.98996751, -11.88817468, -18.48591361,
       -21.55036361, -15.10870915, -23.88744618])

In [44]:
Var_list_6m = []
for t in range(len(row_sums_per_ticker_6m)):
    return_ticker_6m = row_sums_per_ticker_6m[t]
    Var_list_6m.append(return_ticker_6m)

ret_list_6m = []
for t in tickers:
    ret_list_6m.append(data[t].tail(7))

tick_loss_list_6=[]
for i in range(len(Var_list_6m)):
    tick_loss_ensemble = tick_loss(alpha=0.05, returns= ret_list_6m[i], var=Var_list_6m[i])
    tick_loss_list_6.append(tick_loss_ensemble)

tick_loss_list_6m=[]
for i in range(len(tick_loss_list_6)):
    tick_loss_list_6m.append(tick_loss_list_6[i]/7)

tick_loss_list_6m

[0.6608634178194606,
 0.4356574524062547,
 0.339881560046141,
 0.5820200629920701,
 0.5827287747963082,
 0.7610041444330589,
 0.665604072105273,
 0.7460935427616942,
 5.942073714254076]

# 9 months

In [46]:
# Computing tick losses and var forecast at horizon=1 for all tickers and all models
tick_losses_all_9m = []
var_forecast_all_9m =[]

for t in tickers:
    # Get tick loss and forecasts for each model
    t_loss_qr, all_var_forecast_qr = rolling_forecast_QR(data, dates9m, t, alpha=0.05, columns_to_exclude=to_exclude3, horizon=9)
    t_loss_garch, all_var_forecast_garch = rolling_forecast_GARCH(data, dates9m, t, alpha=0.05, columns_to_exclude=to_exclude3, horizon=9)
    t_loss_xgboost, all_var_forecast_xgboost = rolling_forecast_XGB(data, t, alpha=0.05, exclude=to_exclude3, dates=dates9m, horizon=9)

    # Combine tick losses into a matrix
    tick_losses = np.column_stack((t_loss_qr, t_loss_garch, t_loss_xgboost))
    tick_losses_all_9m.append(tick_losses)

    # Combine var forecasts into a matrix
    var_forecast = np.column_stack((all_var_forecast_qr,all_var_forecast_garch,all_var_forecast_xgboost))
    var_forecast_all_9m.append(var_forecast)

tick_losses_all_9m = np.array(tick_losses_all_9m)
var_forecast_all_9m = np.array(var_forecast_all_9m)

print("Tick Losses Matrix:")
print(tick_losses_all_9m)
print("VaR Forecasts All:")
print(var_forecast_all_9m)

Iteration:      1,   Func. Count:      6,   Neg. LLF: 874.1785677557702
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1200.4721531726964
Iteration:      3,   Func. Count:     19,   Neg. LLF: 870.6852289593519
Iteration:      4,   Func. Count:     26,   Neg. LLF: 867.8399619926613
Iteration:      5,   Func. Count:     32,   Neg. LLF: 867.828481407204
Iteration:      6,   Func. Count:     37,   Neg. LLF: 867.8058039900275
Iteration:      7,   Func. Count:     42,   Neg. LLF: 867.748173849943
Iteration:      8,   Func. Count:     47,   Neg. LLF: 867.7462745676646
Iteration:      9,   Func. Count:     52,   Neg. LLF: 867.7461706287772
Iteration:     10,   Func. Count:     57,   Neg. LLF: 867.746169625292
Iteration:     11,   Func. Count:     61,   Neg. LLF: 867.7461696251956
Optimization terminated successfully    (Exit mode 0)
            Current function value: 867.746169625292
            Iterations: 11
            Function evaluations: 61
            Gradient evaluations: 11
It

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

Iteration:      1,   Func. Count:      6,   Neg. LLF: 904.2346809763922
Iteration:      2,   Func. Count:     13,   Neg. LLF: 909.8940809007032
Iteration:      3,   Func. Count:     19,   Neg. LLF: 901.9676011216195
Iteration:      4,   Func. Count:     26,   Neg. LLF: 885.2200651421106
Iteration:      5,   Func. Count:     31,   Neg. LLF: 885.2195442810739
Iteration:      6,   Func. Count:     36,   Neg. LLF: 885.2195104425602
Iteration:      7,   Func. Count:     41,   Neg. LLF: 885.2193138255832
Iteration:      8,   Func. Count:     46,   Neg. LLF: 885.2183522300605
Iteration:      9,   Func. Count:     51,   Neg. LLF: 885.2147616595502
Iteration:     10,   Func. Count:     56,   Neg. LLF: 885.2128950597746
Iteration:     11,   Func. Count:     61,   Neg. LLF: 885.2127609855501
Iteration:     12,   Func. Count:     66,   Neg. LLF: 885.2127585731427
Iteration:     13,   Func. Count:     70,   Neg. LLF: 885.2127585730582
Optimization terminated successfully    (Exit mode 0)
          

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

Iteration:      1,   Func. Count:      6,   Neg. LLF: 3737.673074947868
Iteration:      2,   Func. Count:     14,   Neg. LLF: 1059.3458407887865
Iteration:      3,   Func. Count:     20,   Neg. LLF: 990.3442313922687
Iteration:      4,   Func. Count:     26,   Neg. LLF: 774.7400448569347
Iteration:      5,   Func. Count:     32,   Neg. LLF: 779.060699049972
Iteration:      6,   Func. Count:     38,   Neg. LLF: 771.3832501664328
Iteration:      7,   Func. Count:     44,   Neg. LLF: 769.7415752103974
Iteration:      8,   Func. Count:     50,   Neg. LLF: 765.6335026311029
Iteration:      9,   Func. Count:     55,   Neg. LLF: 765.6297294381544
Iteration:     10,   Func. Count:     60,   Neg. LLF: 765.6289482309151
Iteration:     11,   Func. Count:     65,   Neg. LLF: 765.628625130922
Iteration:     12,   Func. Count:     70,   Neg. LLF: 765.6283320636917
Iteration:     13,   Func. Count:     75,   Neg. LLF: 765.6283000831261
Iteration:     14,   Func. Count:     80,   Neg. LLF: 765.6282984

...

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1210.4827289658347
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1011.3744844764094
Iteration:      3,   Func. Count:     19,   Neg. LLF: 1009.5684780724663
Iteration:      4,   Func. Count:     24,   Neg. LLF: 1009.5683778425991
Iteration:      5,   Func. Count:     29,   Neg. LLF: 1009.5682735267333
Iteration:      6,   Func. Count:     34,   Neg. LLF: 1009.5680105828636
Iteration:      7,   Func. Count:     39,   Neg. LLF: 1009.5663903221539
Iteration:      8,   Func. Count:     44,   Neg. LLF: 1009.5628278473197
Iteration:      9,   Func. Count:     49,   Neg. LLF: 1009.562807759047
Iteration:     10,   Func. Count:     53,   Neg. LLF: 1009.5628077585338
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1009.562807759047
            Iterations: 10
            Function evaluations: 53
            Gradient evaluations: 10
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1215.021

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

Tick Losses Matrix:
[[[0.67870466 0.61092777 0.55985757]]

 [[0.95651844 0.3782583  0.39782924]]

 [[1.14474722 0.28075914 0.27755664]]

 [[1.53189965 0.85915659 0.92248251]]

 [[0.43332927 0.52113312 0.54909767]]

 [[0.67705845 0.64295052 0.60827535]]

 [[2.0188016  0.54095685 0.70002638]]

 [[0.40272198 0.45083194 0.25043074]]

 [[2.5811165  0.97828768 0.92985191]]]
VaR Forecasts All:
[[[-14.06886188  -9.15195024  -8.75467099]
  [ -1.33593183  -8.89037173  -7.22538793]
  [ -7.02467762  -8.66274921  -7.93711955]
  [-18.82430606  -9.12655521  -7.82883134]]

 [[ -1.89707402  -9.73720522 -10.03641086]
  [-11.2559395   -9.61872766 -10.99602953]
  [-18.64531688  -9.89763858 -10.99602911]
  [ -6.28086004 -11.18766598  -9.97844279]]

 [[ -3.17493931  -6.68612522  -6.13305759]
  [ -7.44986454  -6.55071659  -5.46979735]
  [-10.43970095  -6.46193831  -5.46979614]
  [-13.69116956  -6.78137166  -7.4701428 ]]

 [[ -5.77484755 -11.71477732  -8.85425241]
  [  1.01487031 -11.01852195  -8.56716685]
  

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(


In [47]:
tick_losses_all_9m = np.array(tick_losses_all_9m)
tick_losses_all_9m_matrix = np.reshape(tick_losses_all_9m, (9, 3))
tick_losses_all_9m_matrix

array([[0.67870466, 0.61092777, 0.55985757],
       [0.95651844, 0.3782583 , 0.39782924],
       [1.14474722, 0.28075914, 0.27755664],
       [1.53189965, 0.85915659, 0.92248251],
       [0.43332927, 0.52113312, 0.54909767],
       [0.67705845, 0.64295052, 0.60827535],
       [2.0188016 , 0.54095685, 0.70002638],
       [0.40272198, 0.45083194, 0.25043074],
       [2.5811165 , 0.97828768, 0.92985191]])

In [48]:

# Compute weights for each particular ticker with higher weight to lower tick loss models
weights_per_ticker_9m = []
for tick_losses in tick_losses_all_9m_matrix:
    inverted_tick_losses = 1 / tick_losses
    total_inverted_loss = np.sum(inverted_tick_losses)
    weights = inverted_tick_losses / total_inverted_loss
    weights_per_ticker_9m.append(weights)

In [49]:
# Print the weights for each particular ticker
for ticker, weights in enumerate(weights_per_ticker_9m):
    print(f"Weights for Ticker {ticker+1}: {weights}")

Weights for Ticker 1: [0.3009128  0.33429634 0.36479086]
Weights for Ticker 2: [0.16854629 0.42621042 0.40524329]
Weights for Ticker 3: [0.10867555 0.4431059  0.44821855]
Weights for Ticker 4: [0.22503999 0.40125245 0.37370756]
Weights for Ticker 5: [0.3815804  0.31728928 0.30113032]
Weights for Ticker 6: [0.3158432  0.33259839 0.3515584 ]
Weights for Ticker 7: [0.13130593 0.49002174 0.37867233]
Weights for Ticker 8: [0.28559977 0.25512235 0.45927789]
Weights for Ticker 9: [0.15590315 0.41133524 0.4327616 ]


In [50]:
weights_array_9m = np.array(weights_per_ticker_9m)

In [51]:
# Multiply each VaR forecast matrix by the corresponding weights
weighted_var_forecasts_9m = []
row_sums_per_ticker_9m = []
for ticker in range(len(var_forecast_all_9m)):
    weights = weights_array_9m[ticker]
    var_matrix = var_forecast_all_9m[ticker]
    weighted_var_matrix = var_matrix * np.transpose(weights)
    weighted_var_forecasts_9m.append(weighted_var_matrix)
    row_sums_9m = np.sum(weighted_var_matrix, axis=1)
    row_sums_per_ticker_9m.append(row_sums_9m)

# Print the weighted VaR forecast matrices and their row sums (weithed ensemble predictions)
for ticker, (weighted_var_matrix, row_sums_9m) in enumerate(zip(weighted_var_forecasts_9m, row_sums_per_ticker_9m)):
    print(f"Weighted VaR Forecast Matrix for Ticker {ticker+1}:")
    print(weighted_var_matrix)
    print(f"Row sums for Ticker {ticker+1}:")
    print(row_sums_9m)

Weighted VaR Forecast Matrix for Ticker 1:
[[-4.23350064 -3.0594635  -3.19362392]
 [-0.40199899 -2.97201876 -2.63575544]
 [-2.11381542 -2.89592538 -2.89538863]
 [-5.66447467 -3.05097403 -2.85588608]]
Row sums for Ticker 1:
[-10.48658806  -6.00977319  -7.90512943 -11.57133478]
Weighted VaR Forecast Matrix for Ticker 2:
[[-0.31974478 -4.15009834 -4.06718819]
 [-1.89714679 -4.09960197 -4.45606722]
 [-3.14259891 -4.21847671 -4.45606705]
 [-1.05861563 -4.76829983 -4.04369702]]
Row sums for Ticker 2:
[ -8.53703131 -10.45281598 -11.81714266  -9.87061248]
Weighted VaR Forecast Matrix for Ticker 3:
[[-0.34503828 -2.96266155 -2.74895017]
 [-0.80961813 -2.90266119 -2.45166462]
 [-1.13454025 -2.86332301 -2.45166408]
 [-1.48789539 -3.00486581 -3.34825656]]
Row sums for Ticker 3:
[-6.05664999 -6.16394394 -6.44952733 -7.84101775]
Weighted VaR Forecast Matrix for Ticker 4:
[[-1.29957165 -4.70058309 -3.30890105]
 [ 0.22838641 -4.42120892 -3.201615  ]
 [-1.50437717 -4.13901085 -3.201615  ]
 [-2.32720832

In [53]:
for t in range(len(row_sums_per_ticker_9m)):
    return_ticker_9m = row_sums_per_ticker_9m[t]
    print(f"Ensemble forecasts for Ticker {t+1}:",return_ticker_9m)

Ensemble forecasts for Ticker 1: [-10.48658806  -6.00977319  -7.90512943 -11.57133478]
Ensemble forecasts for Ticker 2: [ -8.53703131 -10.45281598 -11.81714266  -9.87061248]
Ensemble forecasts for Ticker 3: [-6.05664999 -6.16394394 -6.44952733 -7.84101775]
Ensemble forecasts for Ticker 4: [-9.30905579 -7.39443752 -8.84500302 -9.88323852]
Ensemble forecasts for Ticker 5: [-10.68114297  -8.29835049  -7.16762011  -9.5778491 ]
Ensemble forecasts for Ticker 6: [ -8.50595443  -6.8172229   -7.01837489 -12.6464079 ]
Ensemble forecasts for Ticker 7: [-6.03844417 -7.14999966 -7.04911399 -8.32692649]
Ensemble forecasts for Ticker 8: [-10.12474509  -9.84200992  -9.5344365  -11.74060319]
Ensemble forecasts for Ticker 9: [-17.57726789 -18.80749143 -15.22392612 -13.02229151]


In [54]:
return_ticker_9m.flatten()

array([-17.57726789, -18.80749143, -15.22392612, -13.02229151])

In [57]:
Var_list_9m = []
for t in range(len(row_sums_per_ticker_9m)):
    return_ticker_9m = row_sums_per_ticker_9m[t]
    Var_list_9m.append(return_ticker_9m)

ret_list_9m = []
for t in tickers:
    ret_list_9m.append(data[t].tail(4))

tick_loss_list_9=[]
for i in range(len(Var_list_9m)):
    tick_loss_ensemble = tick_loss(alpha=0.05, returns= ret_list_9m[i], var=Var_list_9m[i])
    tick_loss_list_9.append(tick_loss_ensemble)

tick_loss_list_9m=[]
for i in range(len(tick_loss_list_9)):
    tick_loss_list_9m.append(tick_loss_list_9[i]/4)

tick_loss_list_9m

[0.6126927635022043,
 0.381212865861296,
 0.3212628533785579,
 0.8189165250977353,
 0.4960498670407401,
 0.6415329279474294,
 0.7952418542423298,
 0.34505190741687375,
 0.9678605466994543]

# 12 months

In [58]:
# Computing tick losses and var forecast at horizon=1 for all tickers and all models
tick_losses_all_12m = []
var_forecast_all_12m =[]

for t in tickers:
    # Get tick loss and forecasts for each model
    t_loss_qr, all_var_forecast_qr = rolling_forecast_QR(data, dates12m, t, alpha=0.05, columns_to_exclude=to_exclude3, horizon=12)
    t_loss_garch, all_var_forecast_garch = rolling_forecast_GARCH(data, dates12m, t, alpha=0.05, columns_to_exclude=to_exclude3, horizon=12)
    t_loss_xgboost, all_var_forecast_xgboost = rolling_forecast_XGB(data, t, alpha=0.05, exclude=to_exclude3, dates=dates12m, horizon=12)

    # Combine tick losses into a matrix
    tick_losses = np.column_stack((t_loss_qr, t_loss_garch, t_loss_xgboost))
    tick_losses_all_12m.append(tick_losses)

    # Combine var forecasts into a matrix
    var_forecast = np.column_stack((all_var_forecast_qr,all_var_forecast_garch,all_var_forecast_xgboost))
    var_forecast_all_12m.append(var_forecast)

tick_losses_all_12m = np.array(tick_losses_all_12m)
var_forecast_all_12m = np.array(var_forecast_all_12m)

print("Tick Losses Matrix:")
print(tick_losses_all_12m)
print("VaR Forecasts All:")
print(var_forecast_all_12m)

Iteration:      1,   Func. Count:      6,   Neg. LLF: 874.1785677557702
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1200.4721531726964
Iteration:      3,   Func. Count:     19,   Neg. LLF: 870.6852289593519
Iteration:      4,   Func. Count:     26,   Neg. LLF: 867.8399619926613
Iteration:      5,   Func. Count:     32,   Neg. LLF: 867.828481407204
Iteration:      6,   Func. Count:     37,   Neg. LLF: 867.8058039900275
Iteration:      7,   Func. Count:     42,   Neg. LLF: 867.748173849943
Iteration:      8,   Func. Count:     47,   Neg. LLF: 867.7462745676646
Iteration:      9,   Func. Count:     52,   Neg. LLF: 867.7461706287772
Iteration:     10,   Func. Count:     57,   Neg. LLF: 867.746169625292
Iteration:     11,   Func. Count:     61,   Neg. LLF: 867.7461696251956
Optimization terminated successfully    (Exit mode 0)
            Current function value: 867.746169625292
            Iterations: 11
            Function evaluations: 61
            Gradient evaluations: 11


/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(


Iteration:      1,   Func. Count:      6,   Neg. LLF: 904.2346809763922
Iteration:      2,   Func. Count:     13,   Neg. LLF: 909.8940809007032
Iteration:      3,   Func. Count:     19,   Neg. LLF: 901.9676011216195
Iteration:      4,   Func. Count:     26,   Neg. LLF: 885.2200651421106
Iteration:      5,   Func. Count:     31,   Neg. LLF: 885.2195442810739
Iteration:      6,   Func. Count:     36,   Neg. LLF: 885.2195104425602
Iteration:      7,   Func. Count:     41,   Neg. LLF: 885.2193138255832
Iteration:      8,   Func. Count:     46,   Neg. LLF: 885.2183522300605
Iteration:      9,   Func. Count:     51,   Neg. LLF: 885.2147616595502
Iteration:     10,   Func. Count:     56,   Neg. LLF: 885.2128950597746
Iteration:     11,   Func. Count:     61,   Neg. LLF: 885.2127609855501
Iteration:     12,   Func. Count:     66,   Neg. LLF: 885.2127585731427
Iteration:     13,   Func. Count:     70,   Neg. LLF: 885.2127585730582
Optimization terminated successfully    (Exit mode 0)
          

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(


Iteration:      1,   Func. Count:      6,   Neg. LLF: 3737.673074947868
Iteration:      2,   Func. Count:     14,   Neg. LLF: 1059.3458407887865
Iteration:      3,   Func. Count:     20,   Neg. LLF: 990.3442313922687
Iteration:      4,   Func. Count:     26,   Neg. LLF: 774.7400448569347
Iteration:      5,   Func. Count:     32,   Neg. LLF: 779.060699049972
Iteration:      6,   Func. Count:     38,   Neg. LLF: 771.3832501664328
Iteration:      7,   Func. Count:     44,   Neg. LLF: 769.7415752103974
Iteration:      8,   Func. Count:     50,   Neg. LLF: 765.6335026311029
Iteration:      9,   Func. Count:     55,   Neg. LLF: 765.6297294381544
Iteration:     10,   Func. Count:     60,   Neg. LLF: 765.6289482309151
Iteration:     11,   Func. Count:     65,   Neg. LLF: 765.628625130922
Iteration:     12,   Func. Count:     70,   Neg. LLF: 765.6283320636917
Iteration:     13,   Func. Count:     75,   Neg. LLF: 765.6283000831261
Iteration:     14,   Func. Count:     80,   Neg. LLF: 765.6282984

...

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(


Iteration:      1,   Func. Count:      6,   Neg. LLF: 1210.4827289658347
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1011.3744844764094
Iteration:      3,   Func. Count:     19,   Neg. LLF: 1009.5684780724663
Iteration:      4,   Func. Count:     24,   Neg. LLF: 1009.5683778425991
Iteration:      5,   Func. Count:     29,   Neg. LLF: 1009.5682735267333
Iteration:      6,   Func. Count:     34,   Neg. LLF: 1009.5680105828636
Iteration:      7,   Func. Count:     39,   Neg. LLF: 1009.5663903221539
Iteration:      8,   Func. Count:     44,   Neg. LLF: 1009.5628278473197
Iteration:      9,   Func. Count:     49,   Neg. LLF: 1009.562807759047
Iteration:     10,   Func. Count:     53,   Neg. LLF: 1009.5628077585338
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1009.562807759047
            Iterations: 10
            Function evaluations: 53
            Gradient evaluations: 10


/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


Tick Losses Matrix:
[[[1.02431460e+00 4.48037324e-01 4.16401848e-01]]

 [[1.47364232e-01 3.52100568e-01 4.17017000e-01]]

 [[5.07715941e-01 7.04088091e-02 6.80719819e-02]]

 [[1.27721673e+00 1.01013161e+00 8.67105369e-01]]

 [[2.82024909e-01 2.67118358e-01 3.00559924e-01]]

 [[8.75151756e-01 2.48792450e-01 2.09996885e-01]]

 [[5.30614253e-01 4.73110940e-02 4.47113007e-03]]

 [[1.03930713e+00 5.58866429e-01 3.11094395e-01]]

 [[6.91059214e+00 7.34959782e-01 8.40303374e-01]]]
VaR Forecasts All:
[[[-20.67749581  -9.15195024  -8.51924071]]

 [[ -5.64247852  -9.73720522 -11.03553387]]

 [[-15.43226787  -6.68612522  -6.63938868]]

 [[-17.05647957 -11.71477732  -8.85425241]]

 [[-10.0225866   -9.72445558 -10.39328691]]

 [[-21.47095048  -8.94376436  -8.16785305]]

 [[-16.77393049  -7.10786731  -6.25106803]]

 [[-23.02322294 -13.41440888  -8.4589682 ]]

 [[  5.11214796 -16.86135519 -18.96822702]]]


/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(


In [59]:
tick_losses_all_12m = np.array(tick_losses_all_12m)
tick_losses_all_12m_matrix = np.reshape(tick_losses_all_12m, (9, 3))
tick_losses_all_12m_matrix

array([[1.02431460e+00, 4.48037324e-01, 4.16401848e-01],
       [1.47364232e-01, 3.52100568e-01, 4.17017000e-01],
       [5.07715941e-01, 7.04088091e-02, 6.80719819e-02],
       [1.27721673e+00, 1.01013161e+00, 8.67105369e-01],
       [2.82024909e-01, 2.67118358e-01, 3.00559924e-01],
       [8.75151756e-01, 2.48792450e-01, 2.09996885e-01],
       [5.30614253e-01, 4.73110940e-02, 4.47113007e-03],
       [1.03930713e+00, 5.58866429e-01, 3.11094395e-01],
       [6.91059214e+00, 7.34959782e-01, 8.40303374e-01]])

In [60]:
# Compute weights for each particular ticker with higher weight to lower tick loss models
weights_per_ticker_12m = []
for tick_losses in tick_losses_all_12m_matrix:
    inverted_tick_losses = 1 / tick_losses
    total_inverted_loss = np.sum(inverted_tick_losses)
    weights = inverted_tick_losses / total_inverted_loss
    weights_per_ticker_12m.append(weights)

In [61]:
# Print the weights for each particular ticker
for ticker, weights in enumerate(weights_per_ticker_12m):
    print(f"Weights for Ticker {ticker+1}: {weights}")

Weights for Ticker 1: [0.17402973 0.39787131 0.42809896]
Weights for Ticker 2: [0.56436407 0.23620262 0.1994333 ]
Weights for Ticker 3: [0.0638183  0.46019194 0.47598976]
Weights for Ticker 4: [0.26756767 0.33831424 0.39411809]
Weights for Ticker 5: [0.33398608 0.35262419 0.31338973]
Weights for Ticker 6: [0.1151404  0.40501761 0.47984199]
Weights for Ticker 7: [0.00763994 0.08568521 0.90667485]
Weights for Ticker 8: [0.16127785 0.29992358 0.53879858]
Weights for Ticker 9: [0.05368665 0.50479839 0.44151496]


In [62]:
weights_array_12m = np.array(weights_per_ticker_12m)

In [63]:
# Multiply each VaR forecast matrix by the corresponding weights
weighted_var_forecasts_12m = []
row_sums_per_ticker_12m = []
for ticker in range(len(var_forecast_all_12m)):
    weights = weights_array_12m[ticker]
    var_matrix = var_forecast_all_12m[ticker]
    weighted_var_matrix = var_matrix * np.transpose(weights)
    weighted_var_forecasts_12m.append(weighted_var_matrix)
    row_sums_12m = np.sum(weighted_var_matrix, axis=1)
    row_sums_per_ticker_12m.append(row_sums_12m)

# Print the weighted VaR forecast matrices and their row sums (weithed ensemble predictions)
for ticker, (weighted_var_matrix, row_sums_12m) in enumerate(zip(weighted_var_forecasts_12m, row_sums_per_ticker_12m)):
    print(f"Weighted VaR Forecast Matrix for Ticker {ticker+1}:")
    print(weighted_var_matrix)
    print(f"Row sums for Ticker {ticker+1}:")
    print(row_sums_12m)

Weighted VaR Forecast Matrix for Ticker 1:
[[-3.59849908 -3.64129843 -3.64707807]]
Row sums for Ticker 1:
[-10.88687557]
Weighted VaR Forecast Matrix for Ticker 2:
[[-3.18441215 -2.29995343 -2.20085298]]
Row sums for Ticker 2:
[-7.68521856]
Weighted VaR Forecast Matrix for Ticker 3:
[[-0.98486105 -3.07690094 -3.16028104]]
Row sums for Ticker 3:
[-7.22204303]
Weighted VaR Forecast Matrix for Ticker 4:
[[-4.56376257 -3.96327598 -3.48962102]]
Row sums for Ticker 4:
[-12.01665956]
Weighted VaR Forecast Matrix for Ticker 5:
[[-3.34740442 -3.42907825 -3.25714939]]
Row sums for Ticker 5:
[-10.03363206]
Weighted VaR Forecast Matrix for Ticker 6:
[[-2.47217383 -3.62238206 -3.91927887]]
Row sums for Ticker 6:
[-10.01383475]
Weighted VaR Forecast Matrix for Ticker 7:
[[-0.12815183 -0.60903913 -5.66768615]]
Row sums for Ticker 7:
[-6.4048771]
Weighted VaR Forecast Matrix for Ticker 8:
[[-3.71313584 -4.02329747 -4.55768003]]
Row sums for Ticker 8:
[-12.29411334]
Weighted VaR Forecast Matrix for Tic

In [64]:
for t in range(len(row_sums_per_ticker_12m)):
    return_ticker_12m = row_sums_per_ticker_12m[t]
    print(f"Ensemble forecasts for Ticker {t+1}:",return_ticker_12m)

Ensemble forecasts for Ticker 1: [-10.88687557]
Ensemble forecasts for Ticker 2: [-7.68521856]
Ensemble forecasts for Ticker 3: [-7.22204303]
Ensemble forecasts for Ticker 4: [-12.01665956]
Ensemble forecasts for Ticker 5: [-10.03363206]
Ensemble forecasts for Ticker 6: [-10.01383475]
Ensemble forecasts for Ticker 7: [-6.4048771]
Ensemble forecasts for Ticker 8: [-12.29411334]
Ensemble forecasts for Ticker 9: [-16.61188695]


In [65]:
return_ticker_12m.flatten()

array([-16.61188695])

In [66]:
Var_list_12m = []
for t in range(len(row_sums_per_ticker_12m)):
    return_ticker_12m = row_sums_per_ticker_12m[t]
    Var_list_12m.append(return_ticker_12m)

ret_list_12m = []
for t in tickers:
    ret_list_12m.append(data[t].tail(1))

tick_loss_list_12=[]
for i in range(len(Var_list_12m)):
    tick_loss_ensemble = tick_loss(alpha=0.05, returns= ret_list_12m[i], var=Var_list_12m[i])
    tick_loss_list_12.append(tick_loss_ensemble)

tick_loss_list_12m=[]
for i in range(len(tick_loss_list_12)):
    tick_loss_list_12m.append(tick_loss_list_12[i]/1)

tick_loss_list_12m

[0.5347835907564549,
 0.24950123433638094,
 0.09720469950499129,
 1.0252257266295761,
 0.2825771817292996,
 0.30229596987004026,
 0.01216158351298753,
 0.5028516520327236,
 0.7224863698931765]